<a href="https://colab.research.google.com/github/Sheryar-bit/AI-Integrations/blob/main/FYP_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install -U -q transformers accelerate bitsandbytes peft datasets
!pip install -q opencv-python-headless matplotlib

!rm -rf diffusers
!git clone https://github.com/huggingface/diffusers.git

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-xv7h_ur3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-xv7h_ur3
  Resolved https://github.com/huggingface/diffusers.git to commit 4b843c8430272644807c845e55ad8aaaa9da0d61
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.37.0.dev0-py3-none-any.whl size=4893097 sha256=fe52712fc43f8090ad769c95cb19c6918fd92938235cd1849210a8fa8ec523a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-x1ayx0kn/wheels/23/0f/7d/f97813d265ed0e599a78d83afd4e1925740896ca79b46cccfd
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.36.0
    Uninstalling diffusers-0.36.0:
      Successfully uninstalled diffusers-0.36.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB

In [3]:
from google.colab import drive
import os

drive.mount('/content/drive')

DATASET_PATH = "/content/drive/MyDrive/Logos"
OUTPUT_DIR = "/content/drive/MyDrive/logo_model_output"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import json

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

metadata = []
image_files = [f for f in os.listdir(DATASET_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

print(f"Captioning {len(image_files)} images... this may take a few minutes.")

for filename in image_files:
    img_path = os.path.join(DATASET_PATH, filename)
    raw_image = Image.open(img_path).convert('RGB')

    inputs = processor(raw_image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)

    full_caption = f"{caption}, professional logo style"

    metadata.append({"file_name": filename, "text": full_caption})

with open(os.path.join(DATASET_PATH, 'metadata.jsonl'), 'w') as f:
    for entry in metadata:
        f.write(json.dumps(entry) + "\n")

print("Metadata generated successfully!")

KeyboardInterrupt: 

In [5]:
!accelerate config default

# Start Training
!python /content/diffusers/examples/text_to_image/train_text_to_image_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --train_data_dir="$DATASET_PATH" \
  --dataloader_num_workers=2 \
  --resolution=512 \
  --center_crop \
  --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=1500 \
  --learning_rate=1e-04 \
  --max_grad_norm=1 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=0 \
  --output_dir="$OUTPUT_DIR" \
  --checkpointing_steps=500 \
  --validation_prompt="a minimalist logo for a tech company, clean lines" \
  --seed=42 \
  --mixed_precision="fp16"

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.
2026-01-20 14:37:57.909175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768919877.975097    6673 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768919877.985516    6673 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768919878.065767    6673 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768919878.065811    6673 computation_placer.cc:177] computation placer 

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

# Untrained base model
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.to("cuda")

# trained LoRA weights
pipe.load_lora_weights(OUTPUT_DIR)

prompt = "a modern logo for a Elite Spots Brand. "
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]

image.save("generated_logo.png")
image

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image

model_id = "runwayml/stable-diffusion-v1-5"
prompt = "a modern logo for a Elite Spots Brand"
seed = 42 # Using the exact same seed is the "secret" to a fair test
generator = torch.Generator("cuda").manual_seed(seed)

# UNTRAINED (Base) Model
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
print("Generating image with UNTRAINED model...")
image_untrained = pipe(prompt, generator=generator, num_inference_steps=30).images[0]

#(Trained)
print("Loading your LoRA weights...")
pipe.load_lora_weights(OUTPUT_DIR)
# Reset the generator so the noise starts exactly the same
generator = torch.Generator("cuda").manual_seed(seed)

print("Generating image with TRAINED model...")
image_trained = pipe(prompt, generator=generator, num_inference_steps=30).images[0]


comparison = Image.new('RGB', (1024, 512))
comparison.paste(image_untrained, (0, 0))
comparison.paste(image_trained, (512, 0))

print("Left: Untrained (Base SD 1.5) | Right: Your Trained Logo Model")
comparison.save("comparison_result.png")
display(comparison)

In [ ]:
pipe.set_adapters(["default"], adapter_weights=[0.6])

prompt = "professional logo style, minimalist mountain, vector art"
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.0).images[0]
display(image)